<a href="https://colab.research.google.com/github/YahiaNaiem/Wuzzuf/blob/main/Wuzzuf_scrabing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import json
import requests
from bs4 import BeautifulSoup

def insert_skills(jobs_list):
    all_jobs_data = {}
    for job_search in jobs_list:
        response = requests.get(f"https://wuzzuf.net/search/jobs/?a=hpb&q={job_search}&start=0")
        soup = BeautifulSoup(response.text , "html.parser")  #to use in defining the needed values(Num_of_Jobs)
        #getting the number of jobs to loop on
        num_of_jobs_text = soup.find("strong").get_text()
        Num_of_Jobs = int(num_of_jobs_text.replace(',', ''))
        jops_counter = 1
        page_number = 0
        skills_count = {}
        while jops_counter <= Num_of_Jobs: #loop for the number of jobs in all pages
            response = requests.get(f"https://wuzzuf.net/search/jobs/?a=hpb&q={job_search}&start={page_number}")
            soup = BeautifulSoup(response.text, "html.parser")
            page_jobs = soup.find_all("div", attrs={"class": "css-1gatmva e1v1l3u10"})
        
            for job in page_jobs: #loop for jobs in each page
                jops_counter += 1
                skills_tags = job.find("div", attrs={"class": "css-y4udm8"}).find_all("div", attrs={})[1].find_all("a", attrs={"class": "css-5x9pm1"})[0:4]
                try:
                    skills_text = [skill.get_text().lstrip(' · ') for skill in skills_tags]
                except:
                    continue
        
                # Update skills_count dictionary
                for skill in skills_text:
                    skills_count[skill.lower()] = skills_count.get(skill.lower(), 0) + 1
        
            page_number += 1
            print(f"{jops_counter - 1} jobs were scraped")
            print(f"{page_number} pages were scraped")
        
        sorted_skills = dict(sorted(skills_count.items(), key = lambda item: item[1], reverse = True))
        reduced_dict = {key: value for key, value in sorted_skills.items() if value > 4}
        all_jobs_data[job_search] = reduced_dict
        
    with open('jobs_data.json', 'w') as json_file:
        json.dump(all_jobs_data, json_file, indent=4)

15 jobs were scraped
1 pages were scraped
30 jobs were scraped
2 pages were scraped
45 jobs were scraped
3 pages were scraped
60 jobs were scraped
4 pages were scraped
75 jobs were scraped
5 pages were scraped
84 jobs were scraped
6 pages were scraped
15 jobs were scraped
1 pages were scraped
30 jobs were scraped
2 pages were scraped
45 jobs were scraped
3 pages were scraped
60 jobs were scraped
4 pages were scraped
75 jobs were scraped
5 pages were scraped
90 jobs were scraped
6 pages were scraped
105 jobs were scraped
7 pages were scraped
120 jobs were scraped
8 pages were scraped
135 jobs were scraped
9 pages were scraped
150 jobs were scraped
10 pages were scraped
165 jobs were scraped
11 pages were scraped
180 jobs were scraped
12 pages were scraped
195 jobs were scraped
13 pages were scraped
210 jobs were scraped
14 pages were scraped
225 jobs were scraped
15 pages were scraped
240 jobs were scraped
16 pages were scraped
255 jobs were scraped
17 pages were scraped
270 jobs were s

In [12]:
jobs_list = ['data scientist', 'software engineer']  # List of job searches
insert_skills(jobs_list)

15 jobs were scraped
1 pages were scraped
30 jobs were scraped
2 pages were scraped
45 jobs were scraped
3 pages were scraped
60 jobs were scraped
4 pages were scraped
75 jobs were scraped
5 pages were scraped
90 jobs were scraped
6 pages were scraped
105 jobs were scraped
7 pages were scraped
120 jobs were scraped
8 pages were scraped
135 jobs were scraped
9 pages were scraped
150 jobs were scraped
10 pages were scraped
165 jobs were scraped
11 pages were scraped
180 jobs were scraped
12 pages were scraped
195 jobs were scraped
13 pages were scraped
210 jobs were scraped
14 pages were scraped
225 jobs were scraped
15 pages were scraped
240 jobs were scraped
16 pages were scraped
255 jobs were scraped
17 pages were scraped
270 jobs were scraped
18 pages were scraped
285 jobs were scraped
19 pages were scraped
300 jobs were scraped
20 pages were scraped
315 jobs were scraped
21 pages were scraped
330 jobs were scraped
22 pages were scraped
345 jobs were scraped
23 pages were scraped
360

In [6]:
from fastapi import FastAPI
from typing import Dict
import json
import requests
from bs4 import BeautifulSoup

app = FastAPI()

def scrape_jobs_data(jobs_list):
    all_jobs_data = {}
    for job_search in jobs_list:
        response = requests.get(f"https://wuzzuf.net/search/jobs/?a=hpb&q={job_search}&start=0")
        soup = BeautifulSoup(response.text , "html.parser")  
        num_of_jobs_text = soup.find("strong").get_text()
        Num_of_Jobs = int(num_of_jobs_text.replace(',', ''))
        jops_counter = 1
        page_number = 0
        skills_count = {}
        while jops_counter <= Num_of_Jobs: 
            response = requests.get(f"https://wuzzuf.net/search/jobs/?a=hpb&q={job_search}&start={page_number}")
            soup = BeautifulSoup(response.text, "html.parser")
            page_jobs = soup.find_all("div", attrs={"class": "css-1gatmva e1v1l3u10"})
        
            for job in page_jobs: 
                jops_counter += 1
                skills_tags = job.find("div", attrs={"class": "css-y4udm8"}).find_all("div", attrs={})[1].find_all("a", attrs={"class": "css-5x9pm1"})[0:4]
                try:
                    skills_text = [skill.get_text().lstrip(' · ') for skill in skills_tags]
                except:
                    continue
        
                for skill in skills_text:
                    skills_count[skill.lower()] = skills_count.get(skill.lower(), 0) + 1
        
            page_number += 1
        
        sorted_skills = dict(sorted(skills_count.items(), key = lambda item: item[1], reverse = True))
        reduced_dict = {key: value for key, value in sorted_skills.items() if value > 4}
        all_jobs_data[job_search] = reduced_dict
        
    return all_jobs_data

@app.get("/jobs_data/")
def get_jobs_data():
    # Load data from JSON file or scrape if file not found
    try:
        with open('jobs_data.json', 'r') as json_file:
            jobs_data = json.load(json_file)
    except FileNotFoundError:
        jobs_data = scrape_jobs_data(['data scientist', 'software engineer'])  # Example job searches
        with open('jobs_data.json', 'w') as json_file:
            json.dump(jobs_data, json_file, indent=4)
    
    return jobs_data


In [7]:
jobs_list = ['data scientist', 'software engineer']  # List of job searches
scrape_jobs_data(jobs_list)

{'data scientist': {'computer science': 46,
  'information technology (it)': 45,
  'python': 19,
  'sql': 16,
  'engineering': 15,
  'software development': 9,
  'sales': 6,
  'javascript': 6,
  'analysis': 5,
  'machine learning': 5,
  'accounting': 5,
  'finance': 5,
  'financial analysis': 5,
  'marketing': 5,
  'project management': 5,
  'customer service': 5,
  'sales skills': 5},
 'software engineer': {'information technology (it)': 1604,
  'computer science': 1434,
  'engineering': 1042,
  'sales': 373,
  'software development': 350,
  'project management': 251,
  'sales skills': 246,
  'customer service': 177,
  'autocad': 173,
  'microsoft office': 167,
  'design': 162,
  'software engineering': 158,
  'computer engineering': 134,
  'mechanical engineering': 130,
  'management': 130,
  'sales target': 119,
  'architecture': 116,
  'civil engineering': 108,
  'analysis': 95,
  'marketing': 86,
  'quality control': 84,
  'maintenance': 83,
  'quality assurance': 81,
  'javascrip